# Descarga de datos - GOES-16 (Serie R)

## Descarga de datos

Utilizaremos la librería GOES-DL, cuyo código fuente está disponible en
[https://github.com/wvenialbo/GOES-DL/](https://github.com/wvenialbo/GOES-DL/),
para instalarla o actualizarla hacemos:
````sh
pip install -U goes-dl
````
El paquete ya incluye todas las librerías requeridas para funcionar.

In [1]:
# Install the required packages
!pip install -U goes-dl

### Cargar las librerías para la descarga de datos

#### Importar las librerías necesarias

Importar el localizador de productos primarios (PP) de imágenes (ABI), el gerenciador de la fuente de datos (AWS) y el descargador de datasets.


In [2]:
# Import the locator and datasource according to your desired product
from goesdl.dataset.goes import GOESProductLocatorABIPP
from goesdl.datasource import DatasourceAWS
from goesdl.downloader import Downloader

#### Importar las librerías opcionales

Importar los manejadores de repositorio y de caché para la fuente de datos.

_Esto únicamente será necesario si queremos compartir una única instancia de un manejador, o de ambos, entre distintas fuentes de datos; o cuando se desea mayor control del repositorio o el caché . Para este ejemplo lo haremos a modo de demostración._

In [3]:
# Import the cache and repository handlers
from goesdl.datasource import DatasourceCache
from goesdl.utils import FileRepository

### Instanciar los objetos auxiliares

_Esto únicamente será necesario si queremos compartir una única instancia de un manejador, o de ambos, entre distintas fuentes de datos. Para este ejemplo lo haremos a modo de demostración._

#### Instanciar el manejador de repositorio

El manejador de repositorio provee funcionalidades básicas para interactuar con el sistema de archivos local para almacenar los datos descargados y leer los conjuntos de datos desde los archivos locales. Usa como directorio por defecto el directorio de trabajo actual en el momento de su instanciación; en otras palabras, la instrucción

```python
my_repo = FileRepository()
```

es equivalente a

```python
my_repo = FileRepository(".") # u otra cadena equivalente, "", "./", etc.
```

Si se especifica una ruta donde guardar los archivos, crea un nuevo directorio en la ruta indicada si no existe; cualquier directorio principal faltante de esta ruta se crea según sea necesario.

Un único manejador de repositorio puede proveer servicio a varios gerenciadores de fuente de datos, compartiendo un directorio raíz común. Si no se provee una instancia del manejador de repositorio al gerenciador de fuente de datos, éste creará uno privado implícitamente.

In [4]:
# Set the path to your repository
repo_path = "./20201114T20"

# Instance a repository manager
my_repo = FileRepository(repo_path)

#### Instanciar el manejador de caché

El manejador de caché administra la lista de archivos disponibles de un producto determinado en la fuente de datos; obtener esta lista de los servidores puede consumir mucho tiempo y al ser información estática, en muchos casos, se puede ahorrar tiempo manteniendo una única lista local cuando se quiere acceder por separado a distintos productos en una misma sesión.

Establece por defecto un tiempo infinito de refresco, es decir, no se vacía durante toda la sesión. En otras palabras, la instrucción

```python
my_cache = DatasourceCache()
```

es equivalente a

```python
my_cache = DatasourceCache("+inf")
```

El tiempo de permanencia de una lista de archivos en el caché se mantiene por rango de fechas y productos o carpeta de productos de acuerdo a la estructura con la que se guardan los datos en la fuente; en otras palabras, el caché se vacía selectivamente para las listas que ya superaron el tiempo de refresco estipulado; las listas que aún no superaron el tiempo de permanencia se dejan en memoria.

Si se desea que las listas no permanezcan en la memoria, se debe instanciar un manejador de caché con tiempo de refresco igual a 0 segundos y pasarlo como argumento al gerenciador de fuente de datos; por ejemplo, cuando se descargan datos dentro de un rango histórico de fechas.

Cuando se descargan datos que se actualizan en tiempo real, la tasa de refresco debe establecerse acorde con la menor tasa de refresco de la fuente de datos, es decir un periodo igual o menor que la del producto que se actualiza con más frecuencia.

Como el refresco dela lista se realiza por demanda, es conveniente vaciar el caché explícitamente al terminar todas las operaciones de descarga.

Un único manejador de caché puede servir a varios gerenciadores de fuente de datos, compartiendo una tasa de refresco común. Si no se provee una instancia del manejador de caché al gerenciador de fuente de datos, éste creará uno privado implícitamente.

In [5]:
# Note that GOES-R series datasets are updated every 10 minutes.

# Set the cache refresh time
refresh_time = 600

# Instance a cache manager
my_cache = DatasourceCache(600)

### Instanciar los objetos principales

#### Inicializar el localizador de productos

Un localizador de productos es responsable de:

1. Generar una lista de rutas de carpetas en función de la estructura de directorio y las convenciones de nomenclatura del conjunto de datos, los detalles del producto y un rango de fechas especificado. Las rutas generadas deben cubrir el intervalo de tiempo definido por la granularidad temporal del directorio del conjunto de datos dentro del período requerido; las rutas a las carpetas que contienen las fechas inicial y final deben estar incluidas en la lista.

2. Verificar si un nombre de archivo determinado coincide con el patrón de nombre de archivo del producto en función de las convenciones de nomenclatura del archivo del conjunto de datos y las especificaciones del producto.

3. Extraer la información de "fecha y hora" UTC correspondiente de un nombre de archivo de producto válido.

En este ejemplo, utilizaremos el siguiente producto del instrumento ABI _(Advanced Baseline Imager)_ de los satélites GOES de la serie R (16‒19):

- **CMIP**: _Cloud and Moisture Imagery Product_ (imágenes de los canales básicos)
- **F**: _Full disk_ (escena del disco planetario completo)
- **C13**: _Band 13, Clean longwave infrared window_ (canal 13, IR de “Onda larga limpio”)
-  **G16**: _GOES-16 (GOES East)_ (satélite GOES-16, originalmente GOES-R)

Por defecto se descargan los productos de Nivel 2 (_Level 2_).

Para acceder a más de una banda (o canal) del conjunto de datos de al mismo tiempo, se puede pasar una lista de canales; por ejemplo,

```python
locator = GOESProductLocatorABIPP("CMIP", "F", ["C02", "C13"], "G16")
```

Cuando no se requiere los datos de más de un canal, las siguientes instrucciones

```python
locator = GOESProductLocatorABIPP("CMIP", "F", "C13", "G16")
```

y

```python
locator = GOESProductLocatorABIPP("CMIP", "F", ["C13"], "G16")
```

son equivalentes.

Tenga en cuenta que no todos los conjuntos de datos permiten acceder a más de una banda al mismo tiempo, y que algunos conjuntos de datos pueden incluir varias bandas en el mismo archivo. La convención de llamada en cada caso es específica de cada producto y su localizador; consulte la ayuda o la documentación de la API para obtener más detalles.

In [6]:
# Initialize the product locator for GOES-R Series (set your desired product)
locator = GOESProductLocatorABIPP("CMIP", "F", "C13", "G16")

#### Inicializar el gerenciador de fuente de datos

El gerenciador de fuente de datos es responsable de listar el contenido de un directorio en una ubicación remota y de descargar archivos desde esa ubicación. Requiere como parámetro obligatorio una cadena con la URL del servidor y la ubicación de los archivos de datos; dependiendo del tipo de fuente podría tomar obligatoria u opcionalmente una tupla de cadenas con información adicional, como regiones, etc. En lugar de una cadena también puede recibir un localizador de productos que debe proveer al gerenciador con aquella información.

Opcionalmente recibe un manejador de repositorio, un manejador de caché, o ambos; en caso contrario, crea una instancia implícita de uno de ellos o de ambos.

Para nuestro ejemplo actual, los siguientes códigos son equivalentes:

```python
# Set the cache refresh time
refresh_time = 600

# Instance a cache manager
my_cache = DatasourceCache(refresh_time)

# Initialize the product locator
locator = GOESProductLocatorABIPP("CMIP", "F", "C13", "G16")

# Initialize the datasource manager
datasource = DatasourceAWS(locator, cache=my_cache)
```

o

```python
# Initialize the datasource manager
datasource = DatasourceAWS(locator, cache=refresh_time)
```

o también

```python
# Initialize the datasource manager
datasource = DatasourceAWS(locator, cache=600)
```

Tenga en cuenta que al inicializar el gerenciador, éste intentará conectarse al servidor de la fuente de datos para realizar tareas de verificación de autenticidad e iniciar una sesión, de ser necesario; esto puede tardar varios segundos.

In [7]:
# Initialize the datasource manager
print("Connecting to the datasource...")
datasource = DatasourceAWS(locator, cache=my_cache)
print("Connected!")

Connecting to the datasource...
Connected!


#### Inicializar el gerenciador de descarga de datos

El gerenciador de descarga de datos proporciona métodos para descargar archivos que coincidan con las marcas de tiempo entre una hora de inicio y una hora de finalización desde la fuente de datos. Recibe un objeto gerenciador de fuente de datos y un objeto localizador de productos; opcionalmente puede recibir una cadena de especificación de formato de fecha, una valor booleano indicando si debe imprimir mensajes con el progreso de la descarga, y un entero indicando un valor de toleracia de tiempo, en segundos, para el rango requerido.

El gerenciador de descarga es independiente de la fuente de datos, el conjunto de datos y sus productos. El gerenciador de fuente de datos y el localizador de productos implementen los métodos necesarios especificados en las interfaces `goes.datasource.Datasource` y `goes.dataset.ProductLocator`, respectivamente, para el funcionamiento correcto del gerenciador de descarga.

El valor por defecto de la cadena de reconocimiento de fechas es `"%Y-%m-%dT%H:%M:%S%z"`, estándar para el formato de las marcas de tiempo ISO 8601 sin parte fraccionaria de segundos. Como los datos de los satélites GOES, exceptuando ocasiones particulare, se actualizan cada 10 minutos, podemos dejar de lado el campo de segundos y utilizar un formato ISO 8601 más corto `"%Y-%m-%dT%H:%M%z"`, o cualquier otro formato que sea del agrado o conveniencia del usuario.

Los valores por defecto para la bandera de impresión de mensajes con el progreso de la descarga es verdadero, y el tiempo de toleracia por defecto es de 60 segundos; el valor del tiempo de toleracia se resta de la fecha inicial y se suma a la fecha final del rango de descarga de datos.

Por defecto, el gerenciador de descarga crea una instancia de un manejador de repositorio local que usa el directorio actual para almacenar las descargas.

Los siguientes códigos son equivalentes:

```python
# Set the path to your repository
repo_path = "~/repository/20201114T20"

# Instance a repository manager
my_repo = DatasourceRepository(repo_path)

# Initialize the download manager
downloader = Downloader(
    datasource=datasource,
    locator=locator,
    repository=my_repo,
    date_format=date_format
)
```

o

```python
# Initialize the download manager
downloader = Downloader(
    datasource=datasource,
    locator=locator,
    repository=repo_path,
    date_format=date_format
)
```

o bien,

```python
# Initialize the download manager
downloader = Downloader(
    datasource=datasource,
    locator=locator,
    repository="~/repository/20201114T20",
    date_format=date_format
)
```

In [8]:
# Set the path to your repository
date_format = "%Y-%m-%dT%H:%M%z"
cdate_format = "%Y-%m-%dT%H:%MZ"

# Initialize the download manager
downloader = Downloader(
    datasource=datasource,
    locator=locator,
    repository=my_repo,
    date_format=date_format
)

### Descagar automática de datos

#### Descagar los datos para un único instante

Si no se proporciona un tiempo final, el gerenciador de descarga intentará encontrar los productos requeridos que corresponden a esa único instante de tiempo, asignando el tiempo incial como final y aplicando posteriormente las tolerancias. El tiempo inicial debe especificarse siempre.

La operación retorna una lista de cadenas con la ruta relativa de los archivos descargados respecto a la ruta raíz del repositorio. Si los archivos ya se encuentran en el repositorio, el archivo no será descargado y, en caso que se esté imprimiendo el progreso, esto será indicado. Igualmente, si la lista de archivos aún no ha expirado, se accederá a la copia que está en el caché, de lo contrario se consultará nuevamente con el servidor en busca de actualizaciones.

In [9]:
# Set your desired date...
start_date = "2020-11-14T20:00Z"

# Download the files
files_single_date = downloader.download_files(start=start_date)
print("Download finished!")

Retrieving available file list
1/1 2020/319/20/OR_ABI-L2-CMIPF-M6C13_G16_s20203192000154_e20203192009473_c20203192009573.nc
    ... downloaded succesfully
Download finished!


#### Descagar los datos para un rango de tiempo

Proporcionando un rango de fechas en la forma de un tiempo inicial y un tiempo final, el gerenciador de descarga intentará encontrar los productos que corresponden a ese rango de fechas.

La operación retorna una lista de cadenas con la ruta relativa de los archivos descargados respecto a la ruta raíz del repositorio. Si los archivos ya se encuentran en el repositorio, el archivo no será descargado y, en caso que se esté imprimiendo el progreso, esto será indicado. Igualmente, si la lista de archivos aún no ha expirado, se accederá a la copia que está en el caché, de lo contrario se consultará nuevamente con el servidor en busca de actualizaciones.

In [10]:
# Set your desired date range
start_date = "2020-11-14T20:00Z"
end_date = "2020-11-14T20:10Z"

# Download the files
files_date_range = downloader.download_files(
    start=start_date,
    end=end_date
)
print("Download finished!")

Retrieving available file list
1/2 2020/319/20/OR_ABI-L2-CMIPF-M6C13_G16_s20203192000154_e20203192009473_c20203192009573.nc
    ... already downloaded
2/2 2020/319/20/OR_ABI-L2-CMIPF-M6C13_G16_s20203192010154_e20203192019473_c20203192019581.nc
    ... downloaded succesfully
Download finished!


#### Descagar los datos más recientes

Proporcionando el rango de fechas entre la hora actual y la de hace diez minutos se podrá descargar el último dataset almancenado.

Se puede buscar dentro de los últimos quince minutos ya que, aunque las imágenes se capturan cada diez minutos, el repositorio remoto no siempre se actualiza al instante. Esto puede causar que se descarguen las dos últimas imágenes; pero si el código se coloca en un loop para mantener actualizado el repositorio local, las imágenes que ya se descargaron no se volverán a descargar.


In [11]:
import datetime

# Get the current time in UTC
now = datetime.datetime.now(datetime.UTC)

# Calculate the time 15 minutes ago
fifteen_minutes_ago = now - datetime.timedelta(minutes=15)

# Format the dates in the desired format
start_date = now.strftime(cdate_format)
end_date = fifteen_minutes_ago.strftime(cdate_format)

# Download the files
files_date_range = downloader.download_files(
    start=start_date,
    end=end_date
)
print("Download finished!")

Retrieving available file list
Download finished!


### Descarga selectiva de datos

La implementación actual realiza un montón de filtrados para evitar descargar todos los productos que el usuario no necesita que puedan estar en un mismo directorio del servidor de fuente de datos, así como a un rango parcial de fechas dentro de un mismo directorio; sin embargo, no soporta la opción de filtrar con una granularidad mayor los archivos que corresponden a un rango de tiempo, o con algún otro criterio arbitrario que el usuario requiera.

Para lograr ese cometido, es posible obtener primero la lista de archivos que corresponden al rango y características del producto especificados por el usuario para someterla a un filtrado personalizado antes de continuar con la descarga.

Esto es válido tanto para la descarga de datos para un instante como para un rango de fechas. Para este ejemplo mostraremos únicamente el segundo caso.

#### Descarga de la lista de archivos

Esta operación es idéntica a la descarga automática, con la diferencia de que se invoca al método `list_files()` en lugar de `download_files()`.

La operación retorna una lista de cadenas con la ruta relativa de los archivos descargados respecto a la ruta raíz del repositorio. Si la lista de archivos aún no ha expirado, se accederá a la copia que está en el caché, de lo contrario se consultará nuevamente con el servidor en busca de actualizaciones; sin embargo, ningún archivo será descargado.

In [12]:
# Set your desired date range
start_date = "2020-11-14T20:00Z"
end_date = "2020-11-14T21:00Z"

# Download the file list
files_date_range = downloader.list_files(
    start=start_date,
    end=end_date
)
print("File list retrieved!")

Retrieving available file list
File list retrieved!


#### Filtrado de la lista de archivos

Al obtenerse la lista de archivos, el usuario podrá aplicar el filtro que requiera; para este ejemplo nos limitaremos a seleccionar los primeros 10 archivos de la lista e ignorar el resto.

In [13]:
filtered_date_range = files_date_range[:3]

#### Descarga de los archivos seleccionados

Para realizar esta operación invocamos al método `get_files()` que recibe, como único argumento, nuestra lista de archivos filtrada.

La operación retorna una lista de cadenas con la ruta relativa de los archivos descargados respecto a la ruta raíz del repositorio. Si los archivos ya se encuentran en el repositorio, el archivo no será descargado y, en caso que se esté imprimiendo el progreso, esto será indicado.

In [14]:
# Download the selected files
downloader.get_files(file_paths=filtered_date_range)
print("Download finished!")

1/3 2020/319/20/OR_ABI-L2-CMIPF-M6C13_G16_s20203192000154_e20203192009473_c20203192009573.nc
    ... already downloaded
2/3 2020/319/20/OR_ABI-L2-CMIPF-M6C13_G16_s20203192010154_e20203192019473_c20203192019581.nc
    ... already downloaded
3/3 2020/319/20/OR_ABI-L2-CMIPF-M6C13_G16_s20203192020154_e20203192029473_c20203192029553.nc
    ... downloaded succesfully
Download finished!


**NOTA**: Cualquier error lanzará una excepción, consulte la documentación para obtener información detallada sobre la API y su comportamiento.